# Primeiro rascunho das funções do modelo de difusão

In [8]:
import tensorflow as tf
import tensorflow_datasets as tfds

import os
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
tf.config.set_visible_devices([], 'GPU')

In [9]:
# testar tensforflow:
print(tf.__version__)
tensor = tf.constant([1, 2, 3, 4, 5])
print(tensor)

2.18.0
tf.Tensor([1 2 3 4 5], shape=(5,), dtype=int32)


## 1. Carregar dataset

In [10]:
# Definir constantes (rascunho)
IMG_SIZE = 256  
BATCH_SIZE = 32  
TRAIN_SPLIT = 0.8  
VAL_SPLIT = 0.2  

In [11]:
def load_and_preprocess_image(image, label):
    image = tf.image.resize(image, [IMG_SIZE, IMG_SIZE])
    
    image = tf.cast(image, tf.float32) / 255.0
    return image, label

### load_and_preprocess_image: Redimensiona e normaliza uma imagem. Retorna a imagem normalizada e o label/rotulo
    
- Parâmetros:
    - image: A imagem a ser processada.
    - label: O label associado à imagem.

- (Resize): função tf.cast(image, tf.float32): Converte os valores de pixel da imagem para float. Geralmente, imagens carregadas de arquivos têm valores de pixel inteiros entre 0 e 255. Para redes neurais, aparentemente trabalhar com float32 é a maneira mais comum.

- (Normalize): Divisão por 255.0 (normalização): Após a conversão, cada valor de pixel é dividido por 255.0 para ajustar o intervalo dos valores de pixel de [0, 255] para [0, 1]. 

## 2 Divisão em Conjuntos de Treinamento, Validação e Teste

In [12]:
def split_dataset(dataset, train_split=TRAIN_SPLIT, val_split=VAL_SPLIT):
    # Calcular o número total de exemplos
    total_examples = dataset.cardinality().numpy()
    
    # Calcular o número de exemplos para cada conjunto
    train_size = int(total_examples * train_split)
    val_size = int(total_examples * val_split)
    
    # Dividir o dataset
    train_ds = dataset.take(train_size)
    remaining_ds = dataset.skip(train_size)
    val_ds = remaining_ds.take(val_size)
    
    return train_ds, val_ds

### load_and_preprocess_image: Divide o conjunto de dados em treinamento, validação e teste. Retorna os conjuntos de treino e validação

- Parâmetros:
    - dataset: O conjunto de dados completo.
    - train_split: Proporção de dados para treinamento.
    - val_split: Proporção de dados para validação.

total_examples: A primeira coisa que a função faz é calcular o número total de exemplos presentes no dataset usando a propriedade .cardinality(), que retorna o número de elementos no tf.data.Dataset. Esse valor é convertido para numpy() para ser manipulado como um número inteiro.

## 3. Carregar e Pré-processar a Base de Dados, Dividir Conjuntos e Preparar Pipeline para Treinamento

In [14]:
def prepare_dataset(batch_size=BATCH_SIZE):
    # Carregar o dataset CIFAR-10 (apenas dados de treino)
    dataset, info = tfds.load('cifar10', split='train', as_supervised=True, with_info=True)
    
    # Aplicar pré-processamento em cada exemplo do dataset
    dataset = dataset.map(load_and_preprocess_image, num_parallel_calls=tf.data.AUTOTUNE)
    
    # Dividir o dataset
    train_ds, val_ds = split_dataset(dataset)
    
    # Configurar o conjunto de treinamento e validação com embaralhamento, batch e prefetching -  estudar casos de uso e importância
    train_ds = train_ds.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
    val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)
    
    return train_ds, val_ds

### prepare_dataset: Prepara o dataset para o treinamento, incluindo carregamento, pré-processamento e divisão. Retorna o dataset de treino e o de validação
    
- Parâmetros:
    - batch_size: Tamanho do batch.

- shuffle: O embaralhamento dos dados é uma técnica para garantir que o modelo não aprenda padrões artificiais devido à ordem dos dados. Se os dados forem processados na ordem em que foram carregados, pode haver correlação entre exemplos consecutivos (como, por exemplo, uma sequência de imagens de uma única classe). Isso pode levar a overfitting ou a um treinamento menos eficaz.

- batching: O batching divide o dataset em pequenos batches ou lotes de dados. Cada lote contém um número fixo de exemplos, que são processados juntos durante a fase de treinamento. O batching é essencial para permitir que o modelo seja treinado em grandes volumes de dados sem sobrecarregar a memória.